In [1]:
pip install dfply

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 612 kB 5.0 MB/s 


In [2]:
import os 
import pandas as pd
import numpy as np 
from dfply import *
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Data/big_contest')
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/Data/big_contest


In [5]:
f_name = sorted(os.listdir())
print(f_name)

['loan_result.csv', 'log_data.csv', 'result.csv', 'test.csv', 'train.csv', 'user_spec.csv']


In [6]:
torch.cuda.is_available()

False

train 데이터 불러오기

In [7]:
train = pd.read_csv('train.csv')

In [8]:
train.columns

Index(['Unnamed: 0', 'application_id', 'year_of_work', 'age_cate',
       'credit_score', 'yearly_income', 'employment_type', 'income_type',
       'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'existing_loan_cnt',
       'loanapply_insert_time', 'bank_id', 'product_id', 'loan_limit',
       'loan_rate', 'is_applied'],
      dtype='object')

train 데이터 정리

In [9]:
train = train.drop(['Unnamed: 0'], axis=1)

In [10]:
# 오브젝트 타입의 데이터 추출, 출력
obj = train.select_dtypes(include=['object']).copy()
obj.head(5)

,age_cate,employment_type,income_type,houseown_type,purpose,personal_rehabilitation_yn,loanapply_insert_time
0,20대,계약직,EARNEDINCOME,기타가족소유,BUSINESS,Y,2022-05-21 23:41:10
1,20대,계약직,EARNEDINCOME,기타가족소유,BUSINESS,Y,2022-05-21 23:41:10
2,20대,계약직,EARNEDINCOME,기타가족소유,BUSINESS,Y,2022-05-21 23:41:11
3,20대,계약직,EARNEDINCOME,기타가족소유,BUSINESS,Y,2022-05-21 23:41:09
4,20대,계약직,EARNEDINCOME,기타가족소유,BUSINESS,Y,2022-05-21 23:41:09


In [11]:
# object 변수 중 loanapply_insert_time 제외

train = train.drop(['loanapply_insert_time'], axis=1)

모델 비교

In [12]:
from sklearn.metrics import accuracy_score, f1_score

Xgboost, RandomForest 용 Test, Train 데이터

In [13]:
# age 데이터 수치화 
df = train 
'''
df['age'] = df['age'].replace('10대', '10')
df['age'] = df['age'].replace('20대', '20')
df['age'] = df['age'].replace('30대', '30')
df['age'] = df['age'].replace('40대', '40')
df['age'] = df['age'].replace('50대', '50')
df['age'] = df['age'].replace('60대', '60')
df['age'] = df['age'].replace('70대', '70')
df['age'] = df['age'].replace('80대', '80')
df['age'] = df['age'].replace('90대', '90')


df['age'] = pd.to_numeric(df['age'])
df['age']
'''

"\ndf['age'] = df['age'].replace('10대', '10')\ndf['age'] = df['age'].replace('20대', '20')\ndf['age'] = df['age'].replace('30대', '30')\ndf['age'] = df['age'].replace('40대', '40')\ndf['age'] = df['age'].replace('50대', '50')\ndf['age'] = df['age'].replace('60대', '60')\ndf['age'] = df['age'].replace('70대', '70')\ndf['age'] = df['age'].replace('80대', '80')\ndf['age'] = df['age'].replace('90대', '90')\n\n\ndf['age'] = pd.to_numeric(df['age'])\ndf['age']\n"

In [14]:
# intercome_type 데이터 수치화 
# 원핫 인코딩 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np

def dummy_data(data,columns):
    for column in columns:
        data = pd.concat([data,pd.get_dummies(data[column],prefix = column)], axis=1)
        data = data.drop(column, axis=1)
    return data

dummy_columns = ['income_type', 'houseown_type', 'purpose','age_cate','employment_type','personal_rehabilitation_yn']
df1 = dummy_data(df, dummy_columns)

Train:Test = 8:2

In [15]:
np.random.seed(0)

In [16]:
from sklearn.model_selection import train_test_split

# X 변수, y변수
target = df1["is_applied"]
data = df1.drop(['is_applied'],axis=1)

# Train:Test = 8:2
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=156)

# 1. RandomForest

In [17]:
'''
from sklearn.ensemble import RandomForestClassifier
'''

'\nfrom sklearn.ensemble import RandomForestClassifier\n'

In [18]:
'''
clf = RandomForestClassifier(n_estimators=100, max_depth=3,random_state=0)
clf.fit(X_train,y_train)

clf.preds = clf.predict(X_test)
'''

'\nclf = RandomForestClassifier(n_estimators=100, max_depth=3,random_state=0)\nclf.fit(X_train,y_train)\n\nclf.preds = clf.predict(X_test)\n'

In [19]:
'''
print('정확도:{0:.4f} , F1:{0:.4f}', format(accuracy_score(y_test, clf.preds), f1_score(y_test, clf.preds, average='macro')))
'''

"\nprint('정확도:{0:.4f} , F1:{0:.4f}', format(accuracy_score(y_test, clf.preds), f1_score(y_test, clf.preds, average='macro')))\n"

# 2. XGboost

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10264386 entries, 0 to 10264385
Data columns (total 46 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   application_id                int64  
 1   year_of_work                  float64
 2   credit_score                  float64
 3   yearly_income                 float64
 4   desired_amount                float64
 5   existing_loan_cnt             float64
 6   bank_id                       int64  
 7   product_id                    int64  
 8   loan_limit                    float64
 9   loan_rate                     float64
 10  is_applied                    float64
 11  income_type_EARNEDINCOME      uint8  
 12  income_type_EARNEDINCOME2     uint8  
 13  income_type_FREELANCER        uint8  
 14  income_type_INOCCUPATION      uint8  
 15  income_type_OTHERINCOME       uint8  
 16  income_type_PRACTITIONER      uint8  
 17  income_type_PRIVATEBUSINESS   uint8  
 18  houseown_type_기타가족소유

In [21]:
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [22]:
xgb = XGBClassifier(n_estimators=100, max_depth=3, random_state=0)


In [ ]:
xgb.fit(X_train, y_train)

xgb.preds = xgb.predict(X_test)

[04:05:46] WARNING: /workspace/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


In [ ]:
print('F1 : {0:.4f}'.format(f1_score(y_test,xgb.preds,average='macro')))


# 3. LGBM

In [ ]:
# object 변수 category화
cat_col = train.columns[train.dtypes == 'object']

for i in range(len(cat_col)):
  train[cat_col[i]] = train[cat_col[i]].astype('category')

In [ ]:
np.random.seed(0)

In [ ]:
from sklearn.model_selection import train_test_split

# X 변수, y변수
target = df1["is_applied"]
data = df1.drop(['is_applied'],axis=1)

# Train:Test = 8:2
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=156)

In [ ]:
# ligthgbm 라이브러리

import lightgbm

from lightgbm import LGBMClassifier

In [ ]:
# LightGBM
lgbm = LGBMClassifier(n_estimators=100, max_depth=3,random_state=0)

evals = [(X_test, y_test)]
lgbm.fit(X_train, y_train)

preds = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:,1]

In [ ]:
lgbm.predict

In [ ]:
print('F1 : {0:.4f}'.format(f1_score(y_test,preds,average='macro')))

하이퍼파라미터 튜닝

In [ ]:
"""

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [300, 400],
    'max_depth': [6, 9],
    'min_samples_split': [0.01, 0.05, 0.1],
    'max_features': ['auto', 'sqrt'],
}
"""

In [ ]:
"""
# estimator
estimator = LGBMClassifier()
"""

In [ ]:
"""
from sklearn.model_selection import KFold

kf = KFold(random_state=30,
           n_splits=10,
           shuffle=True,
          )
"""

In [ ]:
"""
# define grid_search
grid_search = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=kf, 
                           n_jobs=-1, 
                           verbose=2
                          )

# fit with (x_train, y_train)
grid_search.fit(X_train, y_train)
"""

최종 학습 모델

In [ ]:
lgbm = LGBMClassifier(n_estimators=0, num_iterations=400, learning_rate=0.8, reg_alpha=0.9, leg_lambda=0.2)

evals = [(X_test, y_test)]
lgbm_n.fit(X_train, y_train, early_stopping_rounds =100, eval_metric = 'logloss', eval_set=evals, verbose=True)

lgbm_preds = lgbm_n.predict(X_test)
lgbm_pred_proba = lgbm_n.predict_proba(X_test)[:,1]

In [ ]:
print('정확도:{0:.4f} , F1:{0:.4f}', format(accuracy_score(y_test, lgbm_preds), f1_score(y_test, lgbm_preds, average='macro')))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from lightgbm import plot_importance


fig, ax = plt.subplots(1,1, figsize=(10, 8))
plot_importance(lgbm_n, ax=ax, max_num_features=20, height=0.4)


fi테스트 데이터 불러오기

In [ ]:
test = pd.read_csv('test.csv')

테스트 데이터 정리

In [ ]:
test['is_applied'] = test['is_applied_x']

In [ ]:
# loanapply_insert_time 제외

test = test.drop(['loanapply_insert_time','is_applied_y', 'is_applied_x','Unnamed: 0'], axis=1)

In [ ]:
cat_col = test.columns[test.dtypes == 'object']

for i in range(len(cat_col)):
  test[cat_col[i]] = test[cat_col[i]].astype('category')

In [ ]:
test.isna().sum()

In [ ]:
# X_df

X_df = test.drop(['is_applied'], axis=1)

In [ ]:
# 모델 적용 

test['is_applied'] = lgbm_n.predict(X_df)

In [ ]:
test.isna().sum()

평가데이터 저장

In [ ]:
final = test[['application_id','product_id','is_applied']]

In [ ]:
final.head()

In [ ]:
final.to_csv('final.csv')